In [ ]:
import geopandas as gpd
from shapely.geometry import Point, Polygon
import lxml.etree as ET
import pandas as pd

def create_polygon(coordinates_str):
    points_list = [tuple(map(float, pair.split(','))) for pair in coordinates_str.split()]
    polygon = Polygon(points_list)
    return gpd.GeoDataFrame(geometry=[polygon])

def create_point(coordinates_str):
    lat, long = map(float, coordinates_str.split(','))
    point = Point(lat, long)
    return gpd.GeoDataFrame(geometry=[point])

In [ ]:
def parse_coordinates(file_path, entity_type, limit):
    entities = []
    with open(file_path, 'r') as f:
        for line in f:
            limit -= 1
            if limit == 0:
                break
            entity = '<' + line.split(":<")[1]
            entity = entity.replace('srsName', 'crsName')
            root = ET.fromstring(entity)
            coordinates = root.findall(".//{http://www.opengis.net/gml}coordinates")
            for coord in coordinates:
                if entity_type == 'polygon':
                    entity_df = create_polygon(coord.text)
                elif entity_type == 'point':
                    entity_df = create_point(coord.text)
                entities.append(entity_df['geometry'][0])
    return entities

In [ ]:
polygon_file = "data/poly1.txt"
point_file = "data/points.txt"
polygon_limit = 10
point_limit = 31

polygons = parse_coordinates(polygon_file, 'polygon', polygon_limit)
points = parse_coordinates(point_file, 'point', point_limit)

print(polygons)
print(points)